# 第五次课-评分作业

**此次作业<span style="color:red;">成绩计入总评</span>，基础题满分10分，选做题满分1分，选做题共两道，分别为1分，选做任意一道即可，若都做则按得分更高的一道给分**

**请将作业命名为<span style="color:red;">HW05-评分作业+姓名+学号.ipynb</span>, 并于<span style="color:red;">3月18日23:59前</span>提交，做选做题的同学请于<span style="color:red;">3月16日23:59前</span>提交，晚于此时间提交的作业，选做题部分不予批改**

**仅提交ipynb文件即可，`client.py`无需提交**

本次作业每部分分值表示为(x+y)pt，其中x为该部分的正常分值，y为该部分中附加题的分值

## 一、多线程服务器（3+1pt）

单线程服务器（参见HW4第三部分）的缺点是它一次只能为一个客户端提供服务，因此，它既不能满足实际情况中同时服务成千上万个客户端的需求；也更容易受到攻击，如恶意客户端连上服务器后什么都不做，但也不关闭连接，导致服务器无法服务其他客户端。

在本部分中，我们将综合正则表达式、多线程并发服务器等技术，模拟多用户同时注册网站的流程。

### 1.1 检验注册信息合法性 (2+1pt)


在注册时，我们要求用户提供以下信息：

- 用户名：不能与已有的用户名重合，只允许包含字母和数字
- 密码：需**同时**包含大写字母、小写字母、数字三种字符，不允许包含其他字符，长度需大于8位
  - <b>（选做题）Task</b>：在函数`UserChecker.passwd_valid`中，我们提供了直接遍历字符串检查密码合法性的代码；请尝试补充当`use_regex`为`True`时，使用正则表达式检查密码合法性的代码<b>（1pt）</b>
- 电话号码：为方便起见，只需是满足首位是1的十一位数字即可
  - **Task**：补充函数`UserChecker.phone_num_valid`的内容<b>（1pt）</b>
- 电子邮件：邮箱中间须有一个`@`，`@`之前由英文字母、数字、短横(`-`)、句点(`.`)组成，且句点和短横既不能为开头，也不能与`@`相邻；为方便起见，`@`之后只能为`test.com`
  - **Task**：补充函数`UserChecker.email_valid`的内容<b>（1pt）</b>

In [5]:
import re

class User:
    def __init__(self, username, passwd, phone_num, email):
        self.username = username 
        self.passwd = passwd
        self.phone_num = phone_num
        self.email = email

    def __eq__(self, other):
        if not isinstance(other, User):
            return False

        return (
            self.username == other.username
            and self.passwd == other.passwd
            and self.phone_num == other.phone_num
            and self.email == other.email
        )

class UserChecker:

    @staticmethod
    def username_valid(username):
        pattern = re.compile(r'^[a-zA-Z0-9]+$') # 不同于[^...]
        # print("1: ",bool(pattern.match(username)))
        return bool(pattern.match(username))
    
    @staticmethod
    def passwd_valid(passwd, use_regex=False):
        if use_regex:
            
            ###############################
            #--- Your code starts here ---#
            ###############################

            pattern = re.compile(r'^(?=.*[a-z])(?=.*[A-Z])(?=.*\d)[A-Za-z\d]{9,}$') # 正向前瞻断言

            ###############################
            #---  Your code ends here  ---#
            ###############################

            return bool(pattern.match(passwd))
        else:
            length_valid = len(passwd) > 8
            if not length_valid:
                return False

            has_upper = False
            has_lower = False
            has_digit = False

            for ch in passwd:
                if ch.isupper():
                    has_upper = True
                elif ch.islower():
                    has_lower = True
                elif ch.isdigit():
                    has_digit = True
                else:
                    return False
            # print('2: ', has_upper and has_lower and has_digit)
            return (has_upper and has_lower and has_digit) 
    
    @staticmethod
    def phone_num_valid(phone_num):

        ###############################
        #--- Your code starts here ---#
        ###############################

        pattern = re.compile(r'^1\d{10}$')

        ###############################
        #---  Your code ends here  ---#
        ###############################

        return bool(pattern.match(phone_num))
    
    @staticmethod
    def email_valid(email):

        ###############################
        #--- Your code starts here ---#
        ###############################

        pattern = re.compile(r'^[a-zA-Z0-9]+([\.\-]?[a-zA-Z0-9]+)*@test\.com$')

        ###############################
        #---  Your code ends here  ---#
        ###############################
        # print('3: ', bool(pattern.match(email)))
        return bool(pattern.match(email))

    def __call__(self, user):
        return (
            self.username_valid(user.username)
            # and self.passwd_valid(user.passwd, True)
            and self.passwd_valid(user.passwd)
            and self.phone_num_valid(user.phone_num) 
            and self.email_valid(user.email)
        )

class UserDatabase:
    def __init__(self):
        self.users = dict()
        self.checker = UserChecker()
    
    def add_user(self, user):
        is_user_valid = self.checker(user) and user.username not in self.users
        # print(is_user_valid)
        if is_user_valid:
            self.users[user.username] = user 
            return True
        else:
            return False
    
    def __len__(self):
        return len(self.users)



以下代码用于测试你的函数实现是否正确

In [6]:
user_db_test = UserDatabase()
# print(user_db_test.add_user(User(
#     username='Carol', passwd='Car01Uninsta11Gensh1n', phone_num='10101010101', email='ca.r-ol@test.com'
# )))
assert user_db_test.add_user(User(
    username='Alice', passwd='This1sAlice', phone_num='12345678901', email='alice.hi@test.com'
))
assert not user_db_test.add_user(User(
    username='Alice', passwd='This1sAnotherAlice', phone_num='12345678901', email='alice.hello@test.com'
))
assert not user_db_test.add_user(User(
    username='Wrong_Username', passwd='This1sAlice', phone_num='12345678901', email='ok@test.com'
))
assert not user_db_test.add_user(User(
    username='PasswdTooShort', passwd='Sh0rt', phone_num='12345678901', email='ok@test.com'
))
assert not user_db_test.add_user(User(
    username='PasswdNoDigit', passwd='NoDigitsHere', phone_num='12345678901', email='ok@test.com'
))
assert not user_db_test.add_user(User(
    username='PasswdNoUpper', passwd='n0upperhere', phone_num='12345678901', email='ok@test.com'
))
assert not user_db_test.add_user(User(
    username='PasswdSpecialChar', passwd='This1sSpecial@', phone_num='12345678901', email='ok@test.com'
))
assert not user_db_test.add_user(User(
    username='Bob', passwd='Bob1sVeryNB', phone_num='notDigit12345678901', email='bob-nb@test.com'
))
assert not user_db_test.add_user(User(
    username='Bob', passwd='BobPhone1sShort', phone_num='1234567890', email='bob-nb@test.com'
))
assert not user_db_test.add_user(User(
    username='Bob', passwd='BobPhone1sLong', phone_num='1234567890123', email='bob-nb@test.com'
))
assert not user_db_test.add_user(User(
    username='Bob', passwd='No1inBobPhone', phone_num='23456789023', email='bob-nb@test.com'
))
assert user_db_test.add_user(User(
    username='Bob', passwd='B0bF1nallySuccess', phone_num='19876543210', email='bob-nb@test.com'
))
assert not user_db_test.add_user(User(
    username='Carol', passwd='Car01PlayGensh1n', phone_num='10101010101', email='carol@tsinghua.edu.cn'
))
assert not user_db_test.add_user(User(
    username='Carol', passwd='Car01PlayGensh1n', phone_num='10101010101', email='carol@test.com '
))
assert not user_db_test.add_user(User(
    username='Carol', passwd='Car01PlayGensh1n', phone_num='10101010101', email='carol@test.com '
))
assert not user_db_test.add_user(User(
    username='Carol', passwd='Car01PlayGensh1n', phone_num='10101010101', email='.carol@test.com '
))
assert not user_db_test.add_user(User(
    username='Carol', passwd='Car01PlayGensh1n', phone_num='10101010101', email='carol-@test.com '
))
assert user_db_test.add_user(User(
    username='Carol', passwd='Car01Uninsta11Gensh1n', phone_num='10101010101', email='ca.r-ol@test.com'
))
assert len(user_db_test) == 3
assert user_db_test.users == {
    'Alice': User(username='Alice', passwd='This1sAlice', phone_num='12345678901', email='alice.hi@test.com'), 
    'Bob': User(username='Bob', passwd='B0bF1nallySuccess', phone_num='19876543210', email='bob-nb@test.com'),
    'Carol': User(username='Carol', passwd='Car01Uninsta11Gensh1n', phone_num='10101010101', email='ca.r-ol@test.com')
}


### 1.2 搭建多线程服务器（1pt）

**Task**: 补全函数`Server.start`中`for`循环内的代码后运行，再运行`HW05-client.py`

**正常的运行结果**
- 服务器注册若干用户后顺利关闭
- 服务器和客户端均无异常/错误

In [6]:
import socket
import threading
import time

class Server:
    def __init__(self):
        self.user_db = UserDatabase()
        self.lock = threading.Lock()

    def start(self):
        server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server.bind(('localhost', 9999))
        server.listen(5)

        all_threads = []
        
        # 通常这里会用while True来保持服务器持续运行
        # 但我们的测试用例只有30组，为了在完成测试后自动关闭服务器，这里使用for循环
        # 思考：假设所有连接的用户都能成功注册，这里能不能写成"while len(self.user_db) < 30:"? 为什么?
        # 答：不能这样写，while len(self.user_db) < 30需要读取UserDataBase中的数据，存在同步问题
        for i in range(30):
            
            client_socket, addr = server.accept()

            ###############################
            #--- Your code starts here ---#
            ###############################

            # TODO: 建立一个新的线程，处理来自客户端的连接，处理函数`handle_client`已经帮你写好 (1pt)
            thread = threading.Thread(target=self.handle_client, args=(i, client_socket))
            all_threads.append(thread)
            thread.start()

            ###############################
            #---  Your code ends here  ---#
            ###############################
        
        for thread in all_threads:
            thread.join()

        server.close()

    def handle_client(self, thread_id, client_socket):

        # 同一个用户可能多次注册，直到注册成功为止
        # 客户端代码逻辑保证: 用户或者最终注册成功，或者最终发送空格字符' '作为username表示放弃注册
        while True:
            data = client_socket.recv(1024).decode('utf-8')
            splitted = data.split(',')
            # 客户端代码逻辑保证: 用户名、密码、电话、邮箱是由','分割的四个字符串
            new_user = User(
                username=splitted[0], passwd=splitted[1], 
                phone_num=splitted[2], email=splitted[3]
            )
            
            # 思考：如果这里不加线程锁，可能会产生什么问题？
            # 答：多个线程可能同时向UserDataBase中同一位置写入数据，导致数据被覆盖
            with self.lock:
                signed_up = self.user_db.add_user(new_user)

                if signed_up:
                    msg_back = 'ok'
                    client_socket.send(msg_back.encode('utf-8')) 

                    print(f'Thread {thread_id}: 用户{splitted[0]}注册成功，当前用户个数为: {len(self.user_db)}')
                    
                    break
                else:
                    msg_back = 'fail'
                    client_socket.send(msg_back.encode('utf-8')) 
                    
                    if splitted[0] == ' ':
                        print(f'Thread {thread_id}: 用户放弃注册')
                        break

                    print(f'Thread {thread_id}: 用户{splitted[0]}注册失败')
                
        client_socket.close()

server = Server()
server.start()

Thread 6: 用户Carol1注册失败
Thread 17: 用户Carol3注册失败
Thread 1: 用户Alice1注册成功，当前用户个数为: 1
Thread 23: 用户Bob0注册失败
Thread 9: 用户Alice2注册成功，当前用户个数为: 2
Thread 2: 用户Carol0注册失败
Thread 4: 用户Bob0注册失败
Thread 0: 用户Bob2注册失败
Thread 11: 用户Carol3注册失败
Thread 28: 用户Carol0注册失败
Thread 14: 用户Alice0注册成功，当前用户个数为: 3
Thread 16: 用户Alice0注册失败
Thread 3: 用户Alice0注册失败
Thread 20: 用户Carol3注册失败
Thread 7: 用户Alice2注册失败
Thread 18: 用户Bob2注册失败
Thread 10: 用户Bob2注册失败
Thread 21: 用户Carol2注册失败
Thread 24: 用户Bob2注册失败
Thread 19: 用户Alice1注册失败
Thread 27: 用户Alice3注册成功，当前用户个数为: 4
Thread 13: 用户Alice2注册失败
Thread 15: 用户Bob2注册失败
Thread 25: 用户Carol0注册失败
Thread 22: 用户Alice3注册失败
Thread 26: 用户Carol2注册失败
Thread 29: 用户Bob3注册失败
Thread 8: 用户Carol3注册失败
Thread 5: 用户Bob0注册失败
Thread 12: 用户Bob2注册失败
Thread 11: 用户Carol3注册失败
Thread 2: 用户Carol0注册失败
Thread 6: 用户Carol1注册失败
Thread 17: 用户Carol3注册失败
Thread 10: 用户Bob2注册失败
Thread 23: 用户Bob0注册失败
Thread 16: 用户放弃注册
Thread 4: 用户Bob0注册失败
Thread 5: 用户Bob0注册失败
Thread 19: 用户放弃注册
Thread 3: 用户放弃注册
Thread 20: 用户Carol3注册失败
Thread 0:

## 二、`yield`初步与`Tkinter`（3pt）

如果你对`yield`和`send`的用法还不熟悉，请先阅读课件上的相关内容，弄懂课件上代码的运行机制。对于不太熟悉代码调试工具的同学，这里也推荐一个网站：[PythonTutor](https://pythontutor.com/)，你可以通过逐行运行代码并随时查看各变量的值来帮助自己阅读代码、理解运行逻辑。

### 2.1 平均数计算器（1.5pt）

**Task**: 使用`yield`完成生成器函数`running_mean`，每次使用`send`向它传递一个值时
- 若为数字(`int`或`float`)，返回它接收到的所有数的平均值和目前接收到的数字个数的元组
  - Hint: 判断是否为数字的一种方法是使用`isinstance`
- 若为`None`，清除之前所有数据并返回`(0.0, 0)`，表明这个平均值计算器被重新初始化
- 否则返回一个字符串`"Wrong input"`，但**无需**清除之前的数据，也**无需**抛出异常

对以上三种情况的处理各占0.5pt
  
以下是一个例子
- 第一步`send(1)`: 返回`(1.0, 1)`
- 第二步`send(0)`: 返回`(0.5, 2)`
- 第三步`send("Hello")`: 返回`"Wrong input"`，并不会抛出异常
- 第四步`send(2)`: 返回`(1.0, 3)`，第三步发送非数字并不会导致计算器初始化，因此返回的是第一、二、四步中给出的数字的均值和个数
- 第五步`send(None)`: 初始化计算器，返回`(0.0, 0)`
- 第六步`send(0.5)`: 返回`(0.5, 1)`

In [7]:
def running_mean():
    
    ###############################
    #--- Your code starts here ---#
    ###############################
   
    sum = 0.0
    count = 0
    recv = None
    while True:
        if recv == None:
            sum = 0.0
            count = 0
            recv = yield (0.0, count)
        elif isinstance(recv, (int ,float)) == False:
            str = 'Wrong input'
            recv = yield str
        else:
            count = count + 1
            sum = sum + recv
            recv = yield (sum / count, count)

    ###############################
    #---  Your code ends here  ---#
    ###############################


以下代码用于检验你的实现是否正确

In [8]:
mean_generator = running_mean()
next(mean_generator)

all_results = []
for num in [1, 0, "Hello", 2, None, 0.5]:
    all_results.append(mean_generator.send(num))

# print(all_results)

# 注：一般来讲，使用 "==" 来判断经运算得到的浮点数与另一浮点数相等是不合适的
# 比如表达式 "0.1 + 0.2 == 0.3" 的结果就是False，这种误差来源于浮点数表示与运算的实现方式（IEEE 754）
# 但测试中涉及的数据 0.0、1.0 和 0.5 都不会产生这种问题，所以这里偷懒直接用了 "==" 来进行判断
assert all_results == [(1.0, 1), (0.5, 2), 'Wrong input', (1.0, 3), (0.0, 0), (0.5, 1)]


### 2.2 使用`Tkinter`实现可交互的平均数计算器（1.5pt）

在本部分中，我们实现一个`Tkinter`界面，包含
- 一个输入框：用于输入数字
- 一个确定按钮：用于提交输入的数字
- 两个输出框：用于显示之前提交数字的平均数和已提交的数字个数

我们将使用在2.1中所实现的平均数计算器

**Task** 完成`RunningMeanCalculator.calculate_running_mean`函数，功能如下
- 读入用户输入的内容（`self.input_entry`）
  - 若为空字符串，则向计算器`send(None)`
  - 若可以被转换为浮点数/整数，则`send`相应的值
  - 否则，`send`这个字符串本身
  - Hint：可以使用`self.input_entry.get()`读取输入的内容
- 输出计算器返回的结果
  - 在`self.output_entry1`输出计算得到的平均数，`self.output_entry2`输出累积数字个数
  - 若用户给的是为非法输入，则在`self.output_entry1`显示`"Wrong input"`，`self.output_entry2`不显示任何内容
  - Hint: 在某个控件上显示给定内容的函数`output_entry_show`已经帮你写好
- 清空`self.input_entry`的内容
  - Hint：可以使用`self.input_entry.delete()`，具体使用方式参见`self.output_entry_show`中的实现
  
对以上三种功能的实现各占0.5pt

In [9]:
import tkinter as tk

class RunningMeanCalculator:
    def __init__(self, root):

        self.running_mean_fn = running_mean()
        next(self.running_mean_fn)

        self.root = root
        self.root.title("Running Mean Calculator")

        self.input_label = tk.Label(self.root, text="输入浮点数:")
        self.input_label.grid(row=0, column=0, pady=5, padx=5, sticky="ew")

        self.input_entry = tk.Entry(self.root)
        self.input_entry.grid(row=0, column=1, pady=5, padx=5, sticky="ew", columnspan=2)

        self.submit_button = tk.Button(self.root, text="确定", command=self.calculate_running_mean)
        self.submit_button.grid(row=0, column=3, pady=5, padx=5, sticky="ew")

        self.output_label1 = tk.Label(self.root, text="累积平均数:")
        self.output_label1.grid(row=1, column=0, pady=5, padx=5, sticky="ew")

        self.output_entry1 = tk.Entry(self.root, state="readonly")
        self.output_entry1.grid(row=1, column=1, pady=5, padx=5, sticky="ew")

        self.output_label2 = tk.Label(self.root, text="数字个数:")
        self.output_label2.grid(row=1, column=2, pady=5, padx=5, sticky="ew")

        self.output_entry2 = tk.Entry(self.root, state="readonly")
        self.output_entry2.grid(row=1, column=3, pady=5, padx=5, sticky="ew")

    def calculate_running_mean(self):

        ###############################
        #--- Your code starts here ---#
        ###############################
        
        # 读入用户输入的内容
        input = self.input_entry.get()
        
        # 判断各种情况
        if input == '': # 重新初始化
            ans = self.running_mean_fn.send(None)
            self.output_entry_show(self.output_entry1, str(ans[0]))
            self.output_entry_show(self.output_entry2, str(ans[1]))
        else:
            try:
                input = float(input) # 计算
                ans = self.running_mean_fn.send(input)
                self.output_entry_show(self.output_entry1, str(ans[0]))
                self.output_entry_show(self.output_entry2, str(ans[1]))
            except ValueError: # 错误输入
                # print('called')
                ans = self.running_mean_fn.send(input)
                self.output_entry_show(self.output_entry1, ans)
                self.output_entry_show(self.output_entry2, '') # 
        
        # 清空输入
        self.input_entry.delete(0, tk.END)

        ###############################
        #---  Your code ends here  ---#
        ###############################
    
    def output_entry_show(self, entry_instance, s):
        '''
        entry_instance：一个只读的tk.Entry()控件
        s: 一个字符串，在entry_instance显示s的内容

        例子：使用self.output_entry_show(self.output_entry1, "123")
        在self.output_label1控件上展示字符串"123"
        '''
        entry_instance.config(state="normal")    # 设为可读写
        entry_instance.delete(0, tk.END)         # 清空内容
        entry_instance.insert(0, s)              # 写入s
        entry_instance.config(state="readonly")  # 设为只读

root = tk.Tk()
app = RunningMeanCalculator(root)
root.mainloop()

2024-03-17 17:47:34.053 python[96567:735938] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


## 三、基于`yield`实现含有控制参数的任务轮转调度（2pt）

请先回顾课件上的相关内容，理解任务轮转调度的基本实现方式

在本题中，每个任务包含总共所需的完成时间`total_time`和每次调度到该任务时的默认工作时间`default_duration`，同时，支持用户定制某几次调度到该任务的工作时间`customized_duration`，以下是一个具体例子

假设某任务`total_time=7`, `default_duration=1`:
- 第0次调度该任务，不指定`customized_duration`：`total_time = total_time - default_duration = 6`
- 第1次调度该任务，指定`customized_duration=2`：`total_time = total_time - customized_duration = 4`
- 第2次调度该任务，不指定`customized_duration`：`total_time = total_time - default_duration = 3`
- 第3次调度该任务，指定`customized_duration=3`：`total_time = total_time - customized_duration = 0`
- 任务完成
  
**Task**
- 完成生成器函数`task1`和`task2`中`while`循环内的代码（1pt）
  - 注：两段代码可以完全相同
- 完成调度函数`RoundRobin`中`while`循环内的代码（1pt）

In [10]:
def task1():
    total_time = 12
    default_duration = 1
    customized_duration = yield

    while total_time > 0:
        actual_duration = 0
        
        ###############################
        #--- Your code starts here ---#
        ###############################

        # TODO:
        # 1. 为actual_duration正确赋值，这个参数的含义是实际工作时间，
        #    也即指定customized_duration时，将customized_duration赋给它
        #    否则将default_duration赋给它
        # 2. 正确计算完成本次调度后的total_time，
        #    若为负数，则调整到0，并同时**修正actual_duration**
        # 例子1：若某次调度时total_time = 3, customized_duration不指定
        #       则调度完成后total_time = 2, actual_duration = 1,
        # 例子2：若某次调度时total_time = 4, customized_duration指定为3
        #        则调度完成后total_time = 1, actual_duration = 3
        # 例子3：若某次调度时total_time = 3, customized_duration指定为4
        #        则调度完成后total_time = 0, actual_duration = 3

        # 1.
        if customized_duration:
            actual_duration = customized_duration
        else:
            actual_duration = default_duration
        
        # 2.
        total_time = max(0, total_time - actual_duration)

        ###############################
        #---  Your code ends here  ---#
        ###############################

        print(f'Working on task1 for time {actual_duration}, ' \
              f'time left {total_time}')
        customized_duration = yield

    print('Task 1 Finished')

def task2():
    total_time = 15
    default_duration = 2
    customized_duration = yield

    while total_time > 0:
        actual_duration = 0

        ###############################
        #--- Your code starts here ---#
        ###############################

        # TODO: 理论上这里的代码与task1()中补充的代码完全相同

        # 1.
        if customized_duration:
            actual_duration = customized_duration
        else:
            actual_duration = default_duration
        
        # 2.
        total_time = max(0, total_time - actual_duration)

        ###############################
        #---  Your code ends here  ---#
        ###############################

        print(f'Working on task2 for time {actual_duration}, ' \
              f'time left {total_time}')
        customized_duration = yield

    print('Task 2 Finished')

def RoundRobin(tasks, customized_times):
    ''' 完成任务轮转调度工作

    参数说明
        - tasks: 一个列表, 包含待调度的任务
        - customized_times: 一个列表, 长度与tasks一致, 为相应的任务指定定制时间
            列表内的每个元素是一个(key-value)为(int-int)的字典
            一组(k-v)表示第k次调度该任务时的定制工作时间customized_duration为v
        例子: 若customized_times[0] = {0:2, 2:4}
              说明第0次调度tasks[0]时指定customized_duration为2
              第2次调度tasks[0]时指定customized_duration为4
              其余时间调度tasks[0]时不指定customized_duration
    '''

    tasks_list = tasks.copy()
    customized_times_list = customized_times.copy()

    i = 0
    while len(tasks_list) > 0:

        print(f'At timestamp {i}:')

        ###############################
        #--- Your code starts here ---#
        ###############################

        # 注：我们已经定义了变量i表示第几次进行任务调度
        j = 0
        while j < len(tasks_list): # 不能用for循环，因为操作了tasks_list
            try:
                if i not in customized_times_list[j].keys():
                    tasks_list[j].send(None)
                else:
                    customized_time = customized_times_list[j][i]
                    tasks_list[j].send(customized_time)
            except StopIteration:
                tasks_list.pop(j)
                customized_times_list.pop(j) # 删掉已完成任务的调度时间表
                continue # 删掉了原来的任务j，j不加1
            j += 1


        ###############################
        #---  Your code ends here  ---#
        ###############################

        i += 1

    print('All tasks finished')

t1 = task1()
next(t1)
t2 = task2()
next(t2)

RoundRobin([t1, t2], [{0: 4, 2: 3, 4: 4}, {1: 3, 2: 3, 3: 1, 6: 1}])

At timestamp 0:
Working on task1 for time 4, time left 8
Working on task2 for time 2, time left 13
At timestamp 1:
Working on task1 for time 1, time left 7
Working on task2 for time 3, time left 10
At timestamp 2:
Working on task1 for time 3, time left 4
Working on task2 for time 3, time left 7
At timestamp 3:
Working on task1 for time 1, time left 3
Working on task2 for time 1, time left 6
At timestamp 4:
Working on task1 for time 4, time left 0
Working on task2 for time 2, time left 4
At timestamp 5:
Task 1 Finished
Working on task2 for time 2, time left 2
At timestamp 6:
Working on task2 for time 1, time left 1
At timestamp 7:
Working on task2 for time 2, time left 0
At timestamp 8:
Task 2 Finished
All tasks finished


参考输出结果如下
```
At timestamp 0:
Working on task1 for time 4, time left 8
Working on task2 for time 2, time left 13
At timestamp 1:
Working on task1 for time 1, time left 7
Working on task2 for time 3, time left 10
At timestamp 2:
Working on task1 for time 3, time left 4
Working on task2 for time 3, time left 7
At timestamp 3:
Working on task1 for time 1, time left 3
Working on task2 for time 1, time left 6
At timestamp 4:
Working on task1 for time 3, time left 0
Working on task2 for time 2, time left 4
At timestamp 5:
Task 1 Finished
Working on task2 for time 2, time left 2
At timestamp 6:
Working on task2 for time 1, time left 1
At timestamp 7:
Working on task2 for time 1, time left 0
At timestamp 8:
Task 2 Finished
All tasks finished
```

## 四、`async/await`基础（2+1pt）

### 4.1 使用`async/await`实现生产者-消费者模型（2pt）

本题代码逻辑与课件上的例子基本一致，请先理解课件上的例子，再完成以下任务

我们用生产者-消费者模型模拟一个产品生产并送到超市的过程，其中
- 生产者为产品工厂，生产出产品并送到转运中心
- 转运中心作为缓冲区，可以用一个`Queue`来模拟
- 消费者为超市，负责接受从转运中心送来的产品

**Task**: 模仿课件上生产者-消费者的例子，完成异步函数`factory`(1pt)和`supermarket`(1pt)

In [13]:
import nest_asyncio
nest_asyncio.apply() 
# 这是为了在Jupyter Notebook中能够顺利运行代码
# 无需理解其具体含义

import asyncio, random

async def factory(transfer_center, products):
    '''
    参数说明：
        - transfer_center: 是一个Queue, 用于模拟转运中心
        - products: 是一个元素为str的列表, 表示工厂所需生产的所有产品名称
    '''

    ###############################
    #--- Your code starts here ---#
    ###############################

    # TODO: 模仿课件上的例子，补充作为生产者的factory的行为
    # 对每一个产品，
    #       1. 生成1~3秒的随机数表示生产用时，使用asyncio.sleep模拟生产过程
    #       2. 打印下面这句话："工厂生产：<产品名字>"
    #       3. 将产品加入到转运中心

    for product in products:
        await asyncio.sleep(random.uniform(1, 3))
        print("工厂生产：<%s>" % product)
        await transfer_center.put(product)

    await transfer_center.put('') # 表示队列结束

    ###############################
    #---  Your code ends here  ---#
    ###############################
    
    print('工厂生产完毕')
    await transfer_center.join()
    

async def supermarket(transfer_center):

    while True:

        ###############################
        #--- Your code starts here ---#
        ###############################

        # TODO: 模仿课件上的例子，补充作为消费者的supermarket的行为
        # 对每一个产品，
        #       1. 生成1~3秒的随机数表示接收用时，使用asyncio.sleep模拟接受过程
        #       2. 打印下面这句话："超市接收：<产品名字>"
        #       3. 从转运中心获取产品
        # 请注意添加退出循环的判断条件
        
        product = await transfer_center.get()
        if product == '':
            transfer_center.task_done()
            break
        await asyncio.sleep(random.uniform(1, 3))
        print("超市接收：<%s>" % product)
        transfer_center.task_done()
        
        ###############################
        #---  Your code ends here  ---#
        ###############################

    print('超市接收完毕')

async def f():
    transfer_center = asyncio.Queue()

    factory_crt = factory(transfer_center, ['商品' + str(i) for i in range(1, 11)])
    supermarket_crt = supermarket(transfer_center)

    await asyncio.gather(factory_crt, supermarket_crt)

asyncio.run(f())

# 以上实现的是工厂生产用时与超市接受用时均值相同时的情况
# 观察输出，可以发现工厂和超市的输出是近似交替出现的
# 可以思考一下当生产用时大于/小于接受用时的时候，输出情况将会如何
# 可以修改上面代码中随机生成时间的范围并运行，验证你的猜想

工厂生产：<商品1>
工厂生产：<商品2>
超市接收：<商品1>
超市接收：<商品2>
工厂生产：<商品3>
超市接收：<商品3>
工厂生产：<商品4>
超市接收：<商品4>
工厂生产：<商品5>
超市接收：<商品5>
工厂生产：<商品6>
工厂生产：<商品7>
超市接收：<商品6>
工厂生产：<商品8>
超市接收：<商品7>
超市接收：<商品8>
工厂生产：<商品9>
超市接收：<商品9>
工厂生产：<商品10>
工厂生产完毕
超市接收：<商品10>
超市接收完毕


以上代码的一种**可能的**输出如下
```
工厂生产：商品1
超市接收：商品1
工厂生产：商品2
工厂生产：商品3
超市接收：商品2
工厂生产：商品4
超市接收：商品3
工厂生产：商品5
超市接收：商品4
工厂生产：商品6
超市接收：商品5
工厂生产：商品7
超市接收：商品6
超市接收：商品7
工厂生产：商品8
工厂生产：商品9
超市接收：商品8
工厂生产：商品10
工厂生产完毕
超市接收：商品9
超市接收：商品10
超市接收完毕
```

### <b>（选做题）</b>4.2 `async/await`与`yield`结合实现异步生成器（1pt）

本题中，我们使用异步生成器模拟一个在超市中购物的过程

我们希望购买15件"商品1"和20件"商品2"，但是初始时，超市货架上只有3件"商品1"和7件"商品2"，因此，我们采取如下策略进行购买（当然在现实生活中大概不会有人采取这种策略的）
- 当某件商品的在货架上的剩余数量足够时，拿走自身需要数量的商品
- 否则，拿走货架上剩余的所有商品后，让超市工作人员对这一商品补货，并前往另一商品的货架
  - 注：工作人员补货的个数是随机的，因此有可能只补一次货后仍然不能满足我们的要求
- 反复执行以上流程，直至所有需要的商品购买完成

**Task**：阅读代码，理解代码运行流程，并补充函数`take_products`中的空缺部分

In [14]:
import nest_asyncio
nest_asyncio.apply()
import asyncio, random

class Product:
    def __init__(self, name):
        self.name = name 
    
    @classmethod
    def produce(cls, num, *args, **kwargs):
        products = []
        for i in range(num):
            products.append(cls.__new__(cls, *args, **kwargs))
        return products

class Supermarket:
    def __init__(self, product_names, product_init_nums):
        self.all_products = {
            product_name: Product.produce(num=product_init_num, name=product_name)
            for product_name, product_init_num in zip(product_names, product_init_nums)
        }

# 为了不与4.1中的函数重名，下面在命名时把supermarket简化成spmarket
async def take_products(spmarket, num, name):
    # print("called")
    # print(num)
    count = 0
    while count < num:
        
        ###############################
        #--- Your code starts here ---#
        ###############################

        # TODO: 当货架上没有名称为name的产品时，让工作人员进行补货
        # 否则获取货架上的一个产品，赋给一个名为变量product作为生成器返回值
        
        if len(spmarket.all_products[name]) == 0:
            await restock_products(spmarket, name)
            # continue # 补完货并没有取货，要继续循环
        # else: # 注释掉else也可以
        product = spmarket.all_products[name].pop(0)
            # print(len(spmarket.all_products[name]))

        ###############################
        #---  Your code ends here  ---#
        ###############################

        count += 1
        print(f'获得第 {count:2d} 个 {name}')
        yield product
        

async def restock_products(spmarket, name):
    # print("called")
    await asyncio.sleep(random.random() * 2 + 1)
    num = random.randint(1, 3)
    spmarket.all_products[name].extend(
        Product.produce(num=num, name=name)
    )
    # print(len(spmarket.all_products[name]))
    print(f'工作人员为 {name} 补货 {num:2d} 件')

async def buy_products(spmarket, num, name):
    async for _ in take_products(spmarket, num, name):
        pass
        
async def f():
    spmarket = Supermarket(
        product_names = ['商品1', '商品2'],
        product_init_nums = [3, 7]
    )
    tasks = [buy_products(spmarket, 15, '商品1'), buy_products(spmarket, 20, '商品2')]
    await asyncio.gather(tasks[0], tasks[1])

asyncio.run(f())


获得第  1 个 商品1
获得第  2 个 商品1
获得第  3 个 商品1
获得第  1 个 商品2
获得第  2 个 商品2
获得第  3 个 商品2
获得第  4 个 商品2
获得第  5 个 商品2
获得第  6 个 商品2
获得第  7 个 商品2
工作人员为 商品1 补货  1 件
获得第  4 个 商品1
工作人员为 商品2 补货  3 件
获得第  8 个 商品2
获得第  9 个 商品2
获得第 10 个 商品2
工作人员为 商品2 补货  3 件
获得第 11 个 商品2
获得第 12 个 商品2
获得第 13 个 商品2
工作人员为 商品1 补货  3 件
获得第  5 个 商品1
获得第  6 个 商品1
获得第  7 个 商品1
工作人员为 商品2 补货  1 件
获得第 14 个 商品2
工作人员为 商品1 补货  1 件
获得第  8 个 商品1
工作人员为 商品2 补货  2 件
获得第 15 个 商品2
获得第 16 个 商品2
工作人员为 商品1 补货  3 件
获得第  9 个 商品1
获得第 10 个 商品1
获得第 11 个 商品1
工作人员为 商品2 补货  1 件
获得第 17 个 商品2
工作人员为 商品1 补货  2 件
获得第 12 个 商品1
获得第 13 个 商品1
工作人员为 商品2 补货  1 件
获得第 18 个 商品2
工作人员为 商品1 补货  2 件
获得第 14 个 商品1
获得第 15 个 商品1
工作人员为 商品2 补货  3 件
获得第 19 个 商品2
获得第 20 个 商品2


以上代码一种**可能的**输出如下
```
获得第  1 个 商品2
获得第  2 个 商品2
获得第  3 个 商品2
获得第  4 个 商品2
获得第  5 个 商品2
获得第  6 个 商品2
获得第  7 个 商品2
获得第  1 个 商品1
获得第  2 个 商品1
获得第  3 个 商品1
工作人员为 商品1 补货  2 件
获得第  4 个 商品1
获得第  5 个 商品1
工作人员为 商品2 补货  3 件
获得第  8 个 商品2
获得第  9 个 商品2
获得第 10 个 商品2
工作人员为 商品1 补货  2 件
获得第  6 个 商品1
获得第  7 个 商品1
工作人员为 商品2 补货  3 件
获得第 11 个 商品2
获得第 12 个 商品2
获得第 13 个 商品2
工作人员为 商品1 补货  1 件
获得第  8 个 商品1
工作人员为 商品2 补货  3 件
获得第 14 个 商品2
获得第 15 个 商品2
获得第 16 个 商品2
工作人员为 商品2 补货  1 件
获得第 17 个 商品2
工作人员为 商品1 补货  1 件
获得第  9 个 商品1
工作人员为 商品2 补货  1 件
获得第 18 个 商品2
工作人员为 商品1 补货  3 件
获得第 10 个 商品1
获得第 11 个 商品1
获得第 12 个 商品1
工作人员为 商品2 补货  3 件
获得第 19 个 商品2
获得第 20 个 商品2
工作人员为 商品1 补货  2 件
获得第 13 个 商品1
获得第 14 个 商品1
工作人员为 商品1 补货  2 件
获得第 15 个 商品1
```